In [ ]:
import pandas as pd
import matplotlib as plt

df = pd.read_csv('D:\\stock\\csv\\9103_20220129205726.csv', encoding='ANSI')

In [ ]:
# 定義進場函數，呼叫範例為(r,b)=inp(df,r,b,i)
def inp(df,r,b,i):
    #r=成本，b=多空方設定，多方=1，空方 = -1
    df['sign'].iloc[i] = b #進場時紀錄多空
    r = r-b*df.iloc[i,1]*1000 #設定多方買進與空方賣出，股票契約乘數1000
    df['trade'].iloc[i]= -b*df.iloc[i,1]*1000 
    return(r,b)
#定義出場函數，呼叫範例為(r,b)=output(df,r,b,price,i)
def outp(df,r,b,price,i):
    #r是資金存量(可知策略報酬)，b是多空方設定，多方=1，空方=-1
    #price=1代表開盤價，price=4 代表收盤價
    r = r+b*df.iloc[i,price]*1000 # 設定多方賣出與空方回補，股票契約乘數1000
    df['ret'].iloc[i] = r #結算紀錄
    df['trade'].iloc[i]= b*df.iloc[i,price]*1000
    r = 0 #歸零
    b = 0 # 多空方歸零
    return(r,b)
def MA(s,l,df):
    df['ma_s']=df.iloc[:,4].rolling(s).mean() #以收盤價計算s日均線(短均線)
    df['ma_l']=df.iloc[:,4].rolling(l).mean() #以收盤價計算l日均線(長均線)
    # 新增 MA 交易訊號欄
    df['ma_sign']=0
    # 設定黃金交叉訊號出現時， 標示=1
    df['ma_sign'][(df['ma_s'].shift(1) < df['ma_l'].shift(1)) &(df['ma_s'] > df['ma_l'])] =1
    # 設定死亡交叉訊號出現時，標示=-1
    df['ma_sign'][(df['ma_s'].shift(1) > df['ma_l'].shift(1)) &(df['ma_s'] < df['ma_l'])] = -1

def RedK(df,hp,i):
    p = 100*(df['收盤價'].iloc[i]-df['開盤價'].iloc[i]) / df['開盤價'].iloc[i] 
    # ma_sp =100*(df['最高價'].iloc[i]-df['ma_s'].iloc[i])/ df['ma_s'].iloc[i]
    if (p > hp) and ( df['收盤價'].iloc[i]>df['ma_s'].iloc[i] ) :
        df['Red_sign'].iloc[i]=1
        return 1
    
    



def RSI(d,df):
    #以今天的收盤價 - 前一天的收盤價，算出一個序列，命名為 X
    x = df.iloc[:,4].diff() #計算前後項資料改變值
    #先將 x 中，小於 0 值設為 0 ，然後每 d 天對 x 取平均
    #再將 x 中， 負值改為正值，然後每 d 天對 x 取平均
    #將兩個平均鄉廚後 * 100，算出一個序列，即為 RSI 值，命名為['rsi']
    #where 函數，用來尋找符合條件的值，並把不符合條件的值指定為其他數值
    df['rsi'] = 100*x.where(x>0,0).rolling(10).mean() / x.where(x>0,-x).rolling(10).mean()
    #新增RSI訊號欄
    df['rsi_sign'] = 0
    #設定RSI指標超賣訊號 = 1 (部位買進)
    df['rsi_sign'][(df['rsi']<30)] = 1
    #設定RSI指標超買訊號 = -1(部位賣出)
    df['rsi_sign'][df['rsi']>70] = -1

def stop(df,lsp,r,b,i):
    #r = 成本，b=多空方設定，多方=1，空方=-1
    mm = r +b*df.iloc[i,4]*1000 #當日結算(收盤價)，股票契約乘數 1000
    mp = mm/(-b*r) #以「當日結算價/進場成本」計算
    #若當日結算比率大於10%或小於5%，則
    if  (mp<lsp) :
        #若符合停利，停損條件，以下一筆開盤價出場
        (r,b)=outp(df,r,b,1,i+1)
    return(r,b)
#計算各項策略績效指標
#df為欲分析策略績效的資料
def result_F(df):
    #計算最後報酬
    last = df['cus'].iloc[-1]
    #計算交易次數
    count = df['sign'][df['sign']!=0].count() #部位紀錄中不等於0的計數
    #計算最大回檔
    def maxdrawdown(s):
        s = s.cummax() - s #歷史最高價 - 現在的序列，cummax 生成當日之前的歷史最高價序列
        return(s.max())
    #使用maxdrawdown函數，參數為'cus'，回傳給mdd
    mdd = maxdrawdown(df['cus'])
    #計算勝率
    #若交易次數=0，則勝率=0
    if count == 0:
        w=0
    else:
        w = df['ret'][df['ret']>0].count()/count
    #將最後報酬，交易次數，最大回檔、勝率，統整成表格
    result =pd.DataFrame({
            '最後報酬':[last],
            '交易次數':[count],
            '最大回檔':[mdd],
            '勝率':[w]
    })
    return result
#輸出回測結果至Excel檔案
#name為指定輸出Excel檔案名稱，df為輸出資料內容，result為績效指標
#K為保留K線數，L為總資料筆數
def out_excel(name,df,result,K,L):
    writer = pd.ExcelWriter('D:\\stock\\result\\' + name +'.xlsx')
    df.to_excel(writer,'0') #將df資料輸出到'0' 工作表
    result.to_excel(writer,'result') #將 result 資料輸出到'result'工作表
    #將 df['cus']資料輸出到'result' 工作表，指定第 5 欄輸出
    df['cus'].to_excel(writer,'result',startcol=5)
    workbook = writer.book
    chart = workbook.add_chart({'type':'line'}) #指定格式建立圖形物件
    chart.add_series({'values':'='+'result'+'!$G$'+str(K+1)+':$G$'+str(L+1),'name':'cus'})
    worksheet = writer.sheets['result']#設定寫出至result工作表
    worksheet.insert_chart('H2',chart)#將圖形插入到工作中
    writer.save()
#進行買賣計算
MA(5,20,df)
RSI(14,df)
K=50 #設定保留K線參數
L=len(df) #取得df資料筆數
r=0 #紀錄交易資金流量
b=0 #設定多空方，多方=1，空方=-1，空手=0
df['sign']=0 #新增的一欄，用來記錄進場多空
df['ret']=0 #新增的一欄，用來記錄出場結算
df['trade']=0
df['Red_sign']=0



In [ ]:
#由於序號從0開始。迴圈從第k-1筆記錄開始執行
for i in range(K-1,L):
    #若 i < 最後一筆，則執行
    if i < L-1:
        #若B=1，表示多方
        if b == 1  :
            #若死亡交叉或RSI賣出訊號出現，則以下一筆開盤價執行多方出場
            if (df['ma_sign'].iloc[i] == -1) or (df['Red_sign'].iloc[i-1] == 1) or (df['rsi_sign'].iloc[i]==1):
                (r,b)=outp(df,r,b,1,i+1)
                # account +=df['trade'].iloc[i]
                # df['account'].iloc[i]=account
            #停利，停損
            (r,b) = stop(df,-0.15,r,b,i)


        # elif b ==-1:
        #     #若黃金交叉或RSI買進訊號出現，則以下一筆開盤價執行空方出場
        #     if (df['ma_sign'].iloc[i] ==1 ) or (df['rsi_sign'].iloc[i]==1):
        #         (r,b)=outp(df,r,b,1,i+1)
        #     #停利，停損
        #     (r,b) = stop(df,0.1,-0.05,r,b,i)
        if b == 0:
            #若黃金交叉或RSI買進訊號出現，則以下一筆開盤價執行多方進場
            if (df['ma_sign'].iloc[i] == 1) or (RedK(df,4,i) == 1): 
                    (r,b)=inp(df,r,1,i+1)

            
            # #若死亡交叉或RSI買進訊號出現，則以下一筆開盤價執行空方進場
            # elif (df['ma_sign'].iloc[i] == -1) or (df['rsi_sign'].iloc[i]== -1):
            #     (r,b)=inp(df,r,-1,i+1)
    #若 i 等於最後一筆，則
    elif i == L-1:
        #若b不等於0(表示持有部位)，則
        if b != 0:
            #以下一筆收盤價出場
            (r,b)=outp(df,r,b,4,i)


In [ ]:
# 計算累計損益
df['cus']=df['ret'].cumsum() #累加'ret' ，賦值給'cus'
df['cus'].plot() #繪圖

In [ ]:
#計算各項策略績效指標
result = result_F(df)
#輸出績效至Excel
out_excel('Report',df,result,K,L)

In [ ]:
pd.set_option('display.max_rows', df.shape[0]+1)
df